In [ ]:
# Restart the session after running this cell (Run it once!)

!git clone https://github.com/sparisi/gym_gridworlds

!pip install ./gym_gridworlds

Cloning into 'gym_gridworlds'...
remote: Enumerating objects: 166, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 166 (delta 22), reused 17 (delta 17), pack-reused 139 (from 1)
Receiving objects: 100% (166/166), 68.22 KiB | 6.82 MiB/s, done.
Resolving deltas: 100% (86/86), done.
Processing ./gym_gridworlds
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 17.5 MB/s eta 0:00:00
  Created wheel for Gym-Gridworlds: filename=Gym_Gridworlds-1.0-py3-none-any.whl size=15432 sha256=3088ff529c94bffc30a4a0b61657f77f4733ed4d50710e5412068c2670df1a96
  Stored in directory: /tmp/pip-ephem-wheel-cache-il2oaym2/wheels/35/b6/53/d1fcfbb39d514897bf6fbae43c1b74a9f3c71ff4f92c6d32d3
Successfully built Gym-Gridworlds


# Import Libraries

In [ ]:
import numpy as np

import torch

import torch.nn as nn

torch._dynamo.config.suppress_errors = True

import torch.optim as optim

import torch.nn.functional as F

import random

from collections import namedtuple, deque

import gym

import gymnasium

from tqdm import trange

import matplotlib.pyplot as plt

import json

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

cuda


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
import warnings

warnings.filterwarnings('ignore')

# Network

In [ ]:
# Define the neural network model

class QNetwork(nn.Module):

    def __init__(self, state_size, action_size, seed, depth, fc_unit, final_layer_unit):

        super().__init__()

        self.seed = torch.manual_seed(seed)

        self.feature_size = 128

        self.feature_layer = nn.Linear(state_size, self.feature_size)

        self.init_weights(self.feature_layer)

        self.layers = nn.ModuleList()

        for i in range(depth):

            if depth == 1:

                self.layers.append(nn.Linear(self.feature_size, final_layer_unit))

            elif i == 0:

                self.layers.append(nn.Linear(self.feature_size, fc_unit))

            elif i == depth - 1:

                self.layers.append(nn.Linear(fc_unit, final_layer_unit))

            else:

                self.layers.append(nn.Linear(fc_unit, fc_unit))



        for layer in self.layers:

          self.init_weights(layer)



        if depth > 0:

          self.final_layer = nn.Linear(final_layer_unit, action_size)

        else:

          self.final_layer = nn.Linear(self.feature_size, action_size)



        #nn.init.zeros_(self.final_layer.weight)

        #nn.init.zeros_(self.final_layer.bias)

        self.init_weights(self.final_layer)



    def init_weights(self, layer):

        nn.init.xavier_normal(layer.weight)

        self.to(device)



    def forward(self, state):

        x = self.feature_layer(state)
        x = F.relu(x)
        for layer in self.layers:
            x = layer(x)
        return self.final_layer(x)


# ReplayBuffer

In [ ]:
class ReplayBuffer:

    def __init__(self, action_size, buffer_size, batch_size, seed):

        self.action_size = action_size

        self.memory = deque(maxlen=buffer_size)

        self.batch_size = batch_size

        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])

        self.seed = random.seed(seed)



    def add(self, state, action, reward, next_state, done):

        e = self.experience(state, action, reward, next_state, done)

        self.memory.append(e)



    def sample(self):

        experiences = random.sample(self.memory, k=self.batch_size)



        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)

        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)

        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)

        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)

        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)



        return (states, actions, rewards, next_states, dones)



    def __len__(self):

        return len(self.memory)

# DQN

In [ ]:
# Define the DQN agent class

class DQNAgent:

    # Initialize the DQN agent

    def __init__(self, state_size, action_size, seed, lr, network):

        self.state_size = state_size

        self.action_size = action_size

        self.seed = random.seed(seed)



        self.qnetwork_local = QNetwork(state_size, action_size, seed, network["depth"], network["fc_units"], network["final_fc_units"]).to(device)

        self.qnetwork_target = QNetwork(state_size, action_size, seed, network["depth"], network["fc_units"], network["final_fc_units"]).to(device)

        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr)



        self.memory = ReplayBuffer(action_size, buffer_size=int(1e5), batch_size=64, seed=seed)

        self.t_step = 0



    def step(self, state, action, reward, next_state, done):

        self.memory.add(state, action, reward, next_state, done)



        self.t_step = (self.t_step + 1) % 4

        if self.t_step == 0:

            if len(self.memory) > 64:

                experiences = self.memory.sample()

                self.learn(experiences, gamma=0.99)



    # Choose an action based on the current state

    def act(self, state, eps=0.):

        #print(state)

        state_tensor = torch.from_numpy(state).float().unsqueeze(0).to(device)

        self.qnetwork_local.eval()

        with torch.no_grad():

            action_values = self.qnetwork_local(state_tensor)

        self.qnetwork_local.train()



        if np.random.random() > eps:

            return action_values.argmax(dim=1).item()

        else:

            return np.random.randint(self.action_size)



    # Learn from batch of experiences

    def learn(self, experiences, gamma):

        states, actions, rewards, next_states, dones = zip(*experiences)

        states = torch.from_numpy(np.vstack(states)).float().to(device)

        actions = torch.from_numpy(np.vstack(actions)).long().to(device)

        rewards = torch.from_numpy(np.vstack(rewards)).float().to(device)

        next_states = torch.from_numpy(np.vstack(next_states)).float().to(device)

        dones = torch.from_numpy(np.vstack(dones).astype(np.uint8)).float().to(device)



        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)

        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))



        Q_expected = self.qnetwork_local(states).gather(1, actions)



        loss = F.mse_loss(Q_expected, Q_targets)

        self.optimizer.zero_grad()

        loss.backward()

        self.optimizer.step()



        self.soft_update(self.qnetwork_local, self.qnetwork_target, tau=1e-3)



        return loss



    def soft_update(self, local_model, target_model, tau):

        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):

            target_param.data.copy_(tau * local_param.data + (1.0 - tau) * target_param.data)

# Hyperparameters

In [ ]:
num_episodes = 500

max_steps_per_episode_train = 150

#max_steps_per_episode_eval = 25

epsilon_start = 1.0

epsilon_end = 0.01

epsilon_decay_rate = 0.99

gamma = 0.99

lr = 1e-4

buffer_size = 10000

batch_size = 128

update_frequency = 3

test_episodes = 1

# Setup Environment

In [ ]:
#env_id = "Gym-Gridworlds/DangerMaze-6x6-v0"

#env_id = "Gym-Gridworlds/Empty-10x10-v0"

#env_id = "Gym-Gridworlds/Penalty-3x3-v0"

#env = gymnasium.make(env_id, render_mode="binary")

#env_eval = gymnasium.make(env_id, max_episode_steps=max_steps_per_episode_eval, render_mode="binary")

env = gym.make('Acrobot-v1')

env_eval = gym.make('Acrobot-v1')

# DQN Agent

In [ ]:
input_dim = env.observation_space.shape[0]

output_dim = env.action_space.n

#depths = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 50]

#fc_units = [500, 366, 262, 215, 187, 168, 153, 142, 133, 125, 87, 54]

#final_units = [500, 366, 263, 217, 188, 165, 157, 144, 134, 135, 75, 54]

depths = [1]

fc_units = [1024]

final_units = [1024]

networks = [QNetwork(input_dim, output_dim, seed=i, depth=depths[i], fc_unit=fc_units[i], final_layer_unit=final_units[i]) for i in range(len(depths))]

netwrok_names = [f"DQN_{depths[i]:02}_{fc_units[i]}_{sum(p.numel() for p in networks[i].parameters())}" for i in range(len(depths))]

data = {}

for network in networks:

  data[network] = {"loss": [], "train_reward": [], "eval_reward": []}



netwrok_names

['DQN_01_1024_107199']

# Evaluation

In [ ]:
def eval(new_agent):

  episode_rewards = []



  action_series = []

  for episode in range(test_episodes):

      action_series.append([])

      state = env_eval.reset()

      episode_reward = 0

      done = False

      while not done:

          action = new_agent.act(state, eps=0.)

          next_state, reward, done, _ = env_eval.step(action)
          #done = term or trunc

          action_series[-1].append(action)

          episode_reward += reward

          state = next_state



      episode_rewards.append(episode_reward)





  average_reward = sum(episode_rewards) / test_episodes

  return average_reward, action_series

# Training

In [ ]:
for seed in range(1):

  print(f"seed = {seed}")

  for i, network in enumerate(networks):

    if i not in [0]:

      continue



    buffer = deque(maxlen=buffer_size)



    new_agent = DQNAgent(input_dim, output_dim, seed=seed, lr = lr, network={"depth": depths[i], "fc_units": fc_units[i], "final_fc_units": final_units[i]})



    # Training



    losses = []

    rewards = []

    average_rewards = []



    ep_mean_reward = 0

    ep_mean_reward_fixed = 0

    ep_mean_loss = 0

    epsilon = epsilon_start



    pbar = trange(num_episodes, position=0, leave=True, desc=f"{netwrok_names[i]}", unit="episode")



    for episode in pbar:

        # Reset the environment

        state = env.reset()

        epsilon = max(epsilon_end, epsilon * epsilon_decay_rate)

        ep_reward = 0

        ep_loss = 0

        #print(state)

        # Run one episode

        for step in range(max_steps_per_episode_train):

            # Choose and perform an action

            action = new_agent.act(state, epsilon)

            next_state, reward, done, _ = env.step(action)
            #done = term or trunc

            #if reward > 0:

            #  print("REWARDDDDDD!")

            buffer.append((state, action, reward, next_state, done))



            #print(len(buffer), batch_size)

            if len(buffer) >= batch_size:

                batch = random.sample(buffer, batch_size)

                # Update the agent's knowledge

                loss = new_agent.learn(batch, gamma)

                ep_loss += loss



            state = next_state



            ep_reward += reward

            # Check if the episode has ended

            if done:

              #print("H")

              break



        rewards.append(ep_reward)

        ep_mean_reward += ep_reward

        if episode % update_frequency == 0:

          ep_mean_reward_fixed = ep_mean_reward

          average_reward, action_series = eval(new_agent)

          ep_mean_reward = 0



        #if episode % (update_frequency * update_frequency) == 0:

        #  eval_Q_plot(new_agent)

          #print(action_series)



        average_rewards.append(average_reward)



        if type(ep_loss) == int:

          record = 0

          pbar.set_postfix(loss=ep_loss, ep_reward=average_reward, train_reward=ep_reward, epsilon=epsilon)

          losses.append(ep_loss)

        else:

          record = 1

          pbar.set_postfix(loss=ep_loss.item(), ep_reward=average_reward, train_reward=ep_reward, epsilon=epsilon)

          losses.append(ep_loss.detach().cpu())



    data[network]["eval_reward"].append(average_rewards)

    if record == 1:

      data[network]["loss"].append(losses)

    data[network]["train_reward"].append(rewards)



    to_be_saved = {"loss": np.array(losses).tolist(), "train_reward": rewards, "eval_reward": average_rewards}

    with open(f"/kaggle/working/Acrobat_{netwrok_names[i][:7]}{seed}.json", "w") as f:

      json.dump(to_be_saved, f)

seed = 0


DQN_01_1024_107199: 100%|██████████| 500/500 [05:15<00:00,  1.58episode/s, ep_reward=-203, epsilon=0.01, loss=147, train_reward=-109]


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/Acrobat_DQN_01_0.json'

In [ ]:
average_rewards

[-500.0,
 -500.0,
 -500.0,
 -113.0,
 -113.0,
 -113.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -112.0,
 -112.0,
 -112.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 -500.0,
 

# Visualization